<a href="https://colab.research.google.com/github/dipanjanS/mastering-intelligent-agents-langgraph-workshop-dhs2025/blob/main/Module-1-Introduction-to-Generative-AI-and-Agentic-AI/M1LC4_Build_a_LLM_powered_Chatbot_with_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a LLM-powered Chatbot in LangGraph

LangGraph is not just a framework to create static graphs. We already know that it can be used for building stateful, agentic applications using LLMs.

We'll now create a simple LLM-powered chatbot using LangGraph. This chatbot will respond directly to user messages.

![](https://i.imgur.com/heeggTe.png)

In [ ]:
!pip install langchain==0.3.27 langchain-openai==0.3.29 langgraph==0.6.5 --quiet

## Enter Open AI API Key & Setup Environment Variables

In [ ]:
from getpass import getpass
import os

OPENAI_KEY = getpass('Enter Open AI API Key: ')
os.environ['OPENAI_API_KEY'] = OPENAI_KEY

## State

First, define the [State](https://langchain-ai.github.io/langgraph/concepts/low_level/#state) of the graph.

The State schema serves as the input schema for all Nodes and Edges in the graph.

Let's use the `TypedDict` class from python's `typing` module as our schema, which provides type hints for the keys.

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

## Create the Nodes, Edges and Graph

In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def chatbot_node(state: State):
    # get the current state (which is basically the input user prompt)
    state = state['messages']
    llm_response = llm.invoke(state)
    return {"messages": [llm_response]} # appending llm_response to messages key

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot_node)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
response = graph.invoke({"messages": "Explain Agentic AI in 2 bullet points"})
response

In [ ]:
print(response['messages'][-1].content)

In [ ]:
from IPython.display import display_markdown

display_markdown(response['messages'][-1].content, raw=True)

## Invoking vs. Streaming in LangGraph

In [ ]:
response = graph.invoke({"messages": "Explain AI in 1 line"})
print(response['messages'][-1].content)

In [ ]:
response = graph.invoke({"messages": "What did we discuss so far?"})
print(response['messages'][-1].content)

In [ ]:
for event in graph.stream({"messages": "Explain AI in 1 line"},
                          stream_mode='values'):
    print(event['messages'])
    print()